In [1]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV, train_test_split

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

from scipy import stats

In [2]:
oldDf = pd.read_csv("Old NSEI.csv")

In [3]:
newDf = pd.read_csv("New NSEI.csv")

In [4]:
def ruleOne(row):
    if row['Open'] > row['Close']:
        return 0
    return 1

def ruleTwo(row):
    if row['Open'] == row['Low']:
        return 0
    elif row['Open'] == row['High']:
        return 1
    elif row['Open'] > np.mean([row['High'], row['Low']]):
        return 0
    return 1

In [5]:
oldDf['Rule 1'] = oldDf.apply(ruleOne, axis=1)
oldDf['Rule 2'] = oldDf.apply(ruleTwo, axis=1)
oldDf['TP'] = oldDf[['High', 'Low', 'Close']].mean(axis=1)
oldDf['Rule 3'] = (oldDf['TP'] < oldDf['TP'].shift(1)).astype(int)
oldDf['classifier'] = stats.mode(oldDf[['Rule 1', 'Rule 2', 'Rule 3']], axis=1).mode

newDf['Rule 1'] = newDf.apply(ruleOne, axis=1)
newDf['Rule 2'] = newDf.apply(ruleTwo, axis=1)
newDf['TP'] = newDf[['High', 'Low', 'Close']].mean(axis=1)
newDf['Rule 3'] = (newDf['TP'] < newDf['TP'].shift(1)).astype(int)
newDf['classifier'] = stats.mode(newDf[['Rule 1', 'Rule 2', 'Rule 3']], axis=1).mode

In [15]:
table = pd.DataFrame(columns=["Accuracy", "Precision", "Recall", "F1"])

In [ ]:
xOld = oldDf[['Open', 'Close', 'High', 'Low']]
yOld = oldDf['classifier']

xNew = newDf[['Open', 'Close', 'High', 'Low']]
yNew = newDf['classifier']

In [18]:
lr = LogisticRegression()
lr.fit(xOld, yOld)
yPred = lr.predict(xNew)

table.loc["LogisticRegression"] = [
    accuracy_score(yNew, yPred),
    precision_score(yNew, yPred),
    recall_score(yNew, yPred),
    f1_score(yNew, yPred)
]

In [19]:
knn = KNeighborsClassifier()
knn.fit(xOld, yOld)
yPred = knn.predict(xNew)

table.loc["KNeighborsClassifier"] = [
    accuracy_score(yNew, yPred),
    precision_score(yNew, yPred),
    recall_score(yNew, yPred),
    f1_score(yNew, yPred)
]

table.head()

,Accuracy,Precision,Recall,F1
LogisticRegression,0.916667,0.823529,1.000000,0.903226
KNeighborsClassifier,0.694444,0.666667,0.428571,0.521739


In [20]:
svc = SVC()
svc.fit(xOld, yOld)
yPred = svc.predict(xNew)

table.loc["SVC"] = [
    accuracy_score(yNew, yPred),
    precision_score(yNew, yPred),
    recall_score(yNew, yPred),
    f1_score(yNew, yPred)
]

table.head()

C:\Users\cw\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,Accuracy,Precision,Recall,F1
LogisticRegression,0.916667,0.823529,1.000000,0.903226
KNeighborsClassifier,0.694444,0.666667,0.428571,0.521739
SVC,0.611111,0.000000,0.000000,0.000000


In [ ]:
tree = DecisionTreeClassifier()
tree.fit(xOld, yOld)
yPred = tree.predict(xNew)

table.loc["DecisionTreeClassifier"] = [
    accuracy_score(yNew, yPred),
    precision_score(yNew, yPred),
    recall_score(yNew, yPred),
    f1_score(yNew, yPred)
]

table.head()

# table.to_csv("Comparative Analysis.csv")

,Accuracy,Precision,Recall,F1
LogisticRegression,0.916667,0.823529,1.000000,0.903226
KNeighborsClassifier,0.694444,0.666667,0.428571,0.521739
SVC,0.611111,0.000000,0.000000,0.000000
DecisionTreeClassifier,0.694444,0.666667,0.428571,0.521739
